# [SETI Breakthrough Listen - E.T. Signal Search](https://www.kaggle.com/c/seti-breakthrough-listen)
>Find extraterrestrial signals in data from deep space 


# Reference
This book was copied from awsaf49's notebook: https://www.kaggle.com/awsaf49/seti-bl-256x256-tfrec-data

Check this amazing notebook, [How To Create TFRecords](https://www.kaggle.com/cdeotte/how-to-create-tfrecords) by [Chris Deotte](https://www.kaggle.com/cdeotte)

# Notes

I noticed that this notebook seems to use float32 instead of float16.
This notebook outputs size 256, 256, but there should be 273 (?) time values.
The original notebook saves snippets 0, 2, and 4 as 3 different 


In this notebook the signal is processed using channel-wise representation of size (256, 256, 3).
Alternately, we could convert channel-wise representation to spatial representations of (3*256, 256, 1).

It appears that version 2 of this notebook had a bug that reshaped the outputs to size (273, 273, 3).  This version was also saved as a dataset that could be used.

Version 3 seems to resolve this issue.

Version 4 included all the snippets and outputs the data in format (546 x 256 x 3) where subsequent ON/OFF snips are concatenated together.

# How to Create TFRecord

In [ ]:
SEED  = 42
FOLDS = 20
# DIM   = 273 # original notebook resized to 256 time values
# IMAGE_SIZE = [273,256];
SNIPS = [0,1,2,3,4,5] # original notebook only used snips [0,2,4]

# Importing Packages

In [ ]:
import numpy as np 
import pandas as pd 
import os, shutil
from glob import glob
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm # a progress bar
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_label_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
test_label_df  = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')

In [ ]:
train_paths = glob('../input/seti-breakthrough-listen/train/**/*.npy')
test_paths = glob('../input/seti-breakthrough-listen/test/**/*.npy')
len(train_paths), len(test_paths)

In [ ]:
train_df = pd.DataFrame({'filepath':train_paths})
train_df['id'] = train_df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
train_df['group'] = train_df.filepath.map(lambda x: x.split('/')[-2])
train_df = pd.merge(train_df, train_label_df, on='id', how='left')
train_df['group_target'] = train_df.group+train_df.target.astype(str)
train_df.head()

In [ ]:
test_df = pd.DataFrame({'filepath':test_paths})
test_df['id'] = test_df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
test_df['group'] = test_df.filepath.map(lambda x: x.split('/')[-2])
test_df.head()

# Check The Data

In [ ]:
train_df.target.value_counts()

In [ ]:
train_df.group.value_counts()

In [ ]:
train_df.groupby(['group','target'])['id'].count()

# Stratified KFold by Groups

KFolds provides train/test indices to split data in train/test sets. It splits dataset into k consecutive folds (without shuffling by default).
Each fold is then used once as a validation while the k - 1 remaining folds form the training set.

Stratified K-Folds provides train/test indices to split data in train/test sets.
This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    train_df.loc[val_idx,'fold'] = fold
# train_df.groupby(['fold', 'target'])['id'].count()

# Check the signals
From the [dataset information](https://www.kaggle.com/c/seti-breakthrough-listen/overview/data-information),
>>
Not all of the “needle” signals look like diagonal lines, and they may not be present for the entirety of all three “A” observations, but what they do have in common is that they are only present in some or all of the “A” observations (panels **1**, **3**, and **5** in the cadence snippets).



In [ ]:
import matplotlib.pyplot as plt, cv2

def load_signal(filepath, snips, dim=None):
    sgnl = np.load(filepath)
    sgnl = sgnl[snips,] # only keep certain snips (on/off/all)
    img  = np.moveaxis(sgnl, 0, -1)
    img  = img.astype(np.float32) # float16 seems to break imencode below
    if dim is not None:
        img = cv2.resize(img, dsize=(dim, dim), interpolation=cv2.INTER_NEAREST)
    return img

def combine_signals(sgnl):
    '''Takes sgnl of type np.array and size (A,B,C) and concatenates snips 0&1, 2&3, and 4&5'''
    '''Returns: np.array of size (2*A,B,C/2)'''
    shape = sgnl.shape
    assert len(shape) == 3
    assert shape[2] % 2 == 0
    return sgnl.reshape(shape[0]*2,shape[1],-1)

def visualize(sgnl):
    sgnl = sgnl.astype(float)
    plt.figure(figsize=(20, 10))
    for idx in range(sgnl.shape[2]):
        plt.subplot(2, 6, idx+1)
        plt.imshow(sgnl[...,idx])
        plt.axis('OFF')
    plt.tight_layout()
    plt.show()

# Visualize Channels

## No Resize

In [ ]:
sgnl = load_signal(train_df[train_df.target==1].filepath.iloc[3], SNIPS, dim=None)
visualize(sgnl)
sgnl.shape, sgnl.dtype

# TFRecord Data

In [ ]:
import tensorflow as tf

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Writing TFRecord (Train)

In [ ]:
def train_serialize_example(feature0, feature1, feature2, feature3):
    feature = {
      'image'         : _bytes_feature(feature0),
      'image_id'      : _bytes_feature(feature1),
      'group'         : _bytes_feature(feature2),    
      'target'        : _int64_feature(feature3),
  }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
train_df.info()

In [ ]:
show = True
folds = train_df.fold.unique().tolist()
for fold in tqdm(folds): # create tfrecord for each fold; # tqdm is a progress bar
    fold_df = train_df[train_df.fold == fold]
    if show:
        print(); print('Writing TFRecord of fold %i :'%(fold))  
    with tf.io.TFRecordWriter('train%.2i-%i.tfrec'%(fold,fold_df.shape[0])) as writer:
        samples = fold_df.shape[0]
#         samples = 200
        it = tqdm(range(samples)) if show else range(samples)
        for k in it: # images in fold
            row = fold_df.iloc[k,:]
            image      = load_signal(row['filepath'], SNIPS, dim=None)
            if len(SNIPS) == 6:
                image      = combine_signals(image)  # combine 6 snips into 3 'supersnips'
            image      = image[...,::-1] # rgb -> bgr, we'll get the rgb form after decoding the tfrec
            image_id   = row['id']
            group      = row['group']
            target     = np.array(row['target'], dtype = np.uint8)
            example  = train_serialize_example(
                cv2.imencode('.png', image)[1].tobytes(),
                str.encode(image_id),
                str.encode(group),
                target,
                )
            writer.write(example)
        if show:
            filepath = 'train%.2i-%i.tfrec'%(fold,fold_df.shape[0])
            filename = filepath.split('/')[-1]
            filesize = os.path.getsize(filepath)/10**6
            print(filename,':',np.around(filesize, 2),'MB')

# Writing TFRecord (Test)

In [ ]:
def test_serialize_example(feature0, feature1, feature2):
    feature = {
      'image'         : _bytes_feature(feature0),
      'image_id'      : _bytes_feature(feature1),
      'group'         : _bytes_feature(feature2),    
  }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
show  = True
folds = 10
l     = int(np.ceil(test_df.shape[0]/folds))
for fold in tqdm(range(folds)): # create tfrecord for each fold
    fold_df = test_df.iloc[l*fold:l*(fold+1)]
    if show:
        print(); print('Writing TFRecord of fold %i :'%(fold))  
    with tf.io.TFRecordWriter('test%.2i-%i.tfrec'%(fold,fold_df.shape[0])) as writer:
        samples = fold_df.shape[0]
        it = tqdm(range(samples)) if show else range(samples)
        for k in it: # images in fold
            row = fold_df.iloc[k,:]
            image      = load_signal(row['filepath'], SNIPS, dim=None)
            if len(SNIPS) == 6:
                image      = combine_signals(image)  # combine 6 snips into 3 'supersnips'
            image      = image[...,::-1] # rgb -> bgr, we'll get the rgb form after decoding the tfrec
            image_id   = row['id']
            group      = row['group']
            example  = test_serialize_example(
                cv2.imencode('.png', image)[1].tobytes(),
                str.encode(image_id),
                str.encode(group),
                )
            writer.write(example)
        if show:
            filepath = 'test%.2i-%i.tfrec'%(fold,fold_df.shape[0])
            filename = filepath.split('/')[-1]
            filesize = os.path.getsize(filepath)/10**6
            print(filename,':',np.around(filesize, 2),'MB')

# Reading TFRecord

In [ ]:
import re, math
def decode_image(image_data):
    image = tf.image.decode_png(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range # could try float16 instead???
    # image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def prepare_target(target):    
    target = tf.cast(target, tf.float32)            
    target = tf.reshape(target, [1])         
    return target

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image" : tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "target": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    # image  = tf.reshape(image, [DIM, DIM, 3])
    target = prepare_target(example['target'])
    return image, target # returns a dataset of (image, label) pairs

def load_dataset(fileids, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(fileids, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset(filenames):
    dataset = load_dataset(filenames, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(20, seed=SEED)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(fileids):
    # the number of data items is written in the id of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(fileid).group(1)) for fileid in fileids]
    return np.sum(n)

# Visual

In [ ]:
def display_batch(batch, size=2):
    imgs, tars = batch
    for img_idx in range(size):
        plt.figure(figsize=(4*2, 12*2))
        for idx in range(3):
            plt.subplot(size, 3, idx+1)
            plt.title(f'Target:{tars[img_idx].numpy()[0]}')
            plt.imshow(imgs[img_idx,:, :, idx])
            plt.text(5, 10, str(idx), bbox={'facecolor': 'white'})
            plt.xticks([])
            plt.yticks([])
        plt.tight_layout()
        plt.show() 

# Total Images

In [ ]:
# INITIALIZE VARIABLES
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob('train*.tfrec') # edited to only fetch "ON" files, but should work either way
TEST_FILENAMES     = tf.io.gfile.glob('test*.tfrec')
print('There are %i train & %i test images'%(count_data_items(TRAINING_FILENAMES), count_data_items(TEST_FILENAMES)))

# Once Batch Image

In [ ]:
# DISPLAY TRAIN IMAGES
training_dataset = get_training_dataset(TRAINING_FILENAMES)
training_dataset = training_dataset.unbatch().batch(20)
train_batch = next(iter(training_dataset))
display_batch(train_batch, 2);

In [ ]:
img, label = train_batch
np.unique(label.numpy(), return_counts=True)